In [0]:
%pip install databricks-sdk --upgrade

In [0]:
CATALOG = dbutils.widgets.get("CATALOG")
REFUNDMANAGER_LAKEBASE_INSTANCE_NAME = f"{CATALOG}refundmanager"

In [ ]:
import sys
sys.path.append('../utils')
from uc_state import add

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import sql

w = WorkspaceClient()

warehouse = w.warehouses.create(
    name=f"{CATALOG}-warehouse",
    cluster_size="2X-Small",
    max_num_clusters=1,
    min_num_clusters=1,
    enable_serverless_compute=True
).result()

add(CATALOG, "warehouses", warehouse)

In [0]:
print(warehouse)

In [0]:
app_yaml_contents = f"""command:
  - uvicorn 
  - app.main:app
env:
  - name: DATABRICKS_WAREHOUSE_ID
    value: '{warehouse.id}'
  - name: DATABRICKS_CATALOG
    value: '{CATALOG}'
"""

# for some reason we need to remove it first before this works
import os
if os.path.exists("../apps/refund-manager/app.yaml"):
    os.remove("../apps/refund-manager/app.yaml")

import time
time.sleep(3)

with open("../apps/refund-manager/app.yaml", "w") as f:
    f.write(app_yaml_contents)

In [0]:
from databricks.sdk.service.apps import App, AppResource, AppResourceSqlWarehouse, AppResourceDatabase, AppResourceSqlWarehouseSqlWarehousePermission, AppResourceDatabaseDatabasePermission
import os

source_code_path = os.path.abspath("../apps/refund-manager")

app = w.apps.create(
    App(
        name=f"{CATALOG}-refundmanager",
        default_source_code_path=source_code_path,
        resources=[
            AppResource(
                name="sql-warehouse",
                sql_warehouse=AppResourceSqlWarehouse(
                    id=warehouse.id, 
                    permission=AppResourceSqlWarehouseSqlWarehousePermission.CAN_USE)                
            ),
            AppResource(
                name="database",
                database=AppResourceDatabase(
                    instance_name=f"{REFUNDMANAGER_LAKEBASE_INSTANCE_NAME}",
                    database_name="caspers",
                    permission=AppResourceDatabaseDatabasePermission.CAN_CONNECT_AND_CREATE
                )
            )
        ]
    )
)

In [0]:
# blocks until app is ready
app_status = app.result()

# add to state
add(CATALOG, "apps", app_status)

In [ ]:
# grant app permissions to catalog, schema, `all_events` table for reading events
from databricks.sdk.service import catalog

w.grants.update(
    full_name=f"{CATALOG}",
    securable_type="CATALOG",
    changes=[
        catalog.PermissionsChange(
            add=[catalog.Privilege.USE_CATALOG],
            principal=app_status.id
        )
    ]
)

w.grants.update(
    full_name=f"{CATALOG}.lakeflow",
    securable_type="SCHEMA",
    changes=[
        catalog.PermissionsChange(
            add=[catalog.Privilege.USE_SCHEMA],
            principal=app_status.id
        )
    ]
)

w.grants.update(
    full_name=f"{CATALOG}.lakeflow.all_events",
    securable_type="TABLE",
    changes=[
        catalog.PermissionsChange(
            add=[catalog.Privilege.SELECT],
            principal=app_status.id
        )
    ]
)

In [0]:
# there's no clean way to update the apps role in postgres so we delete it & recreate it
# TODO: figure this out

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import (
    DatabaseInstanceRole,
    DatabaseInstanceRoleIdentityType,
    DatabaseInstanceRoleMembershipRole,
)

w = WorkspaceClient()

INSTANCE = f"{REFUNDMANAGER_LAKEBASE_INSTANCE_NAME}"
APP_ID  = app_status.id         

current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
w.database.delete_database_instance_role(INSTANCE, APP_ID, reassign_owned_to=current_user)

role = DatabaseInstanceRole(
    name=APP_ID,
    identity_type=DatabaseInstanceRoleIdentityType.SERVICE_PRINCIPAL,
    membership_role=DatabaseInstanceRoleMembershipRole.DATABRICKS_SUPERUSER,
)

# Creates the role
w.database.create_database_instance_role(INSTANCE, role)

In [0]:
app_status

In [0]:
from databricks.sdk.service.apps import AppDeployment

deployment = w.apps.deploy(
    app_name=app_status.name,
    app_deployment=AppDeployment(
        source_code_path=source_code_path
    )
)
deployment_status = deployment.result()
display(deployment_status)